In [1]:
%pip install openai==0.28




[notice] A new release of pip is available: 23.3.2 -> 24.3.1
[notice] To update, run: pip3.10 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:

import json
input_json_file = "input.json"
example_json_content = [
    {
        "segment": 633,
        "start": 1613.107,
        "end": 1623.093,
        "text": "We can't be at each other's throats, so what I'm willing to do is give you 15% off the top, and you provide security and distribution if needed.",
        "audio_file": "SPEAKER_17/segment_633/audio.wav"
    },
    {
        "segment": 634,
        "start": 1623.5,
        "end": 1630.0,
        "text": "That sounds good to me, but is that for the overall production?",
        "audio_file": "SPEAKER_17/segment_634/audio.wav"
    },
       {
        "segment": 635,
        "start": 1630.1,
        "end": 1637.5,
        "text":"It's raining cats and dogs",
        "audio_file": "SPEAKER_17/segment_635/audio.wav"
    },
      {
        "segment": 636,
        "start": 1637.5,
        "end": 1640.0,
        "text": "Getting this deal done is a piece of cake",
        "audio_file": "SPEAKER_17/segment_636/audio.wav"
    }

    ]
with open (input_json_file,"w") as f:
    json.dump(example_json_content,f, indent=4)

In [3]:
import os
import openai
import json

# ====== CONFIGURATION ======
openai.api_key = "sk-YourAPIKey"  # Set your OpenAI API key here
model_name = "gpt-3.5-turbo"            # Model to use for translation
target_language = "French"               # Change to your desired target language
input_file_path = "diar_simple.json"         # Input JSON file path
output_file_path = "example_translated.json"  # Output JSON file path
# ==========================

def translate_text(text, target_language):
    """
    Translates the given text into the target_language using OpenAI's ChatCompletion.
    The system instruction includes a directive for detecting and preserving nuances
    (idiomatic expressions, cultural references, etc.).
    """
    try:
        # We use ChatCompletion with a system message to instruct the model
        # on how we want the translation to be performed.
        response = openai.ChatCompletion.create(
            model=model_name,
            messages=[
                {
                    "role": "system",
                    "content": (
                        "You are a highly skilled translator. "
                        "Your task is to detect the source language automatically and translate it into "
                        f"{target_language} while preserving context, nuances, cultural references, wordplay, "
                        "idiomatic expressions, acronyms, measurement units, and slang terms. "
                        "If there's wordplay or idiomatic expressions that need adaptation, do so while keeping "
                        "the same intent and style."
                    )
                },
                {
                    "role": "user",
                    "content": text
                }
            ],
            temperature=0.2  # Lower temperature for more consistent translations
        )

        # Extract the translated text from the response
        translated_text = response["choices"][0]["message"]["content"].strip()
        return translated_text
    
    except Exception as e:
        print(f"Error during translation: {e}")
        return text  # Fallback to original text if there's an error

def main():
    # 1. Read the input JSON file
    with open(input_file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    # 2. Iterate over each item in the JSON list and translate the "text" field
    for item in data:
        original_text = item.get("text", "")
        
        # Provide an option for manual translation
        print(f"\nOriginal text: {original_text}")
        #user_input = input(
            #"Press ENTER to auto-translate OR type/paste your manual translation here:\n> "
        #).strip()
        
        #if user_input:
            # If user provided a manual translation, use it
            #translated_text = user_input
        #else:
            # Otherwise, auto-translate using OpenAI
        translated_text = translate_text(original_text, target_language)
        
        # Store or update the item with the translated text
        item["translated_text"] = translated_text
    
    # 3. Save the updated data to a new JSON file
    with open(output_file_path, 'w', encoding='utf-8') as f:
        json.dump(data, f, indent=4, ensure_ascii=False)
    
    print(f"\nTranslation complete! Results saved to: {output_file_path}")

if __name__ == "__main__":
    main()



Original text:  Thank you.

Original text:  Typical of Victor Von Doom to build a 30-foot statue of himself.

Original text: Well, it's obviously aimed at first-time visitors to create feelings of smallness.

Original text: Maddox-y.

Original text: Good thing it ain't working.

Original text: Reed, what are we doing here?

Original text: This guy's fast food's stripping all science.

Original text: This wasn't our first stop, in case you forgot.

Original text: Besides, Victor's not that bad.

Original text: He's just a little larger than life.

Original text:  My research suggests that exposure to a high-energy cosmic storm, born on solar winds, might have triggered the evolution of early planetary life.

Original text: In six weeks, another cloud with the same elemental profile will pass Earth's orbit.

Original text: A study conducted in space could fundamentally advance our knowledge about the structure of the human genome, cure countless diseases, extend human life, give kids a 

### Evaluation ( d-Bleu, bert-score, comet-qe)

In [8]:
%pip install sacrebleu


[notice] A new release of pip is available: 23.3.2 -> 24.3.1
[notice] To update, run: pip3.10 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [9]:
import openai
import json
import sacrebleu

Round-trip translation (for evaluation)

In [ ]:


# =========== CONFIGURATION ===========
openai.api_key = "sk-YourAPIKey"  # Set your OpenAI API key here
model_name = "gpt-3.5-turbo"            # Modèle utilisé pour la traduction
input_file = "eval_translated.json"  # Fichier qui contient la traduction
output_file = "backtranslated_3.5.json"
# =====================================

def back_translate(text, source_lang="French", target_lang="English"):
    """
    Effectue la back-translation du texte (source_lang -> target_lang) via OpenAI.
    Ici, on suppose que la traduction existante est en 'French' et que l'original
    était en 'English'. Ajustez si nécessaire.
    """
    try:
        response = openai.ChatCompletion.create(
            model=model_name,
            messages=[
                {
                    "role": "system",
                    "content": (
                        f"Tu es un traducteur expérimenté. "
                        f"Traduis le texte ci-dessous du {source_lang} vers le {target_lang} "
                        f"en préservant le sens et les nuances."
                    )
                },
                {
                    "role": "user",
                    "content": text
                }
            ],
            temperature=0.2
        )
        return response["choices"][0]["message"]["content"].strip()
    except Exception as e:
        print(f"Erreur pendant la back-translation: {e}")
        return text  # Fallback: renvoie le texte d'entrée s'il y a une erreur

def main():
    # 1. Lire le fichier JSON contenant la traduction
    with open(input_file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    # Préparer des listes pour le calcul du score BLEU (corpus-level)
    original_texts = []
    back_translated_texts = []
    
    # 2. Faire la back-translation pour chaque item
    for item in data:
        original_text = item.get("text", "")
        translated_text = item.get("translated_text", "")

        # Back-translation du 'translated_text' vers la langue source (ici, English)
        back_translated = back_translate(translated_text, source_lang="French", target_lang="English")
        
        # On stocke la back-translation dans l'item
        item["back_translated_text"] = back_translated
        
        # On accumule pour calculer un BLEU "corpus" à la fin
        original_texts.append(original_text)
        back_translated_texts.append(back_translated)
    
    # 3. Calculer le score BLEU global (corpus-level)
    #    Note: sacrebleu.corpus_bleu prend en entrée:
    #      - hypotheses: liste des textes hypothèses (ici, back_translated_texts)
    #      - references: liste de listes (ou single reference), ex. [[ref1, ref2, ...], [ref1, ref2, ...], ...]
    #    Donc on encapsule original_texts dans une liste
    bleu = sacrebleu.corpus_bleu(back_translated_texts, [original_texts])
    bleu_score = bleu.score
    
    # 4. Sauvegarder dans un nouveau fichier JSON
    #    On peut ajouter un champ "bleu_score" global à la racine ou l'afficher.
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(data, f, indent=4, ensure_ascii=False)
    
    print(f"Back-translation terminée. Fichier généré : {output_file}")
    print(f"Score BLEU (corpus) = {bleu_score:.2f}")

if __name__ == "__main__":
    main()


In [13]:
input_file = "backtranslated_3.5.json"

In [15]:
def main():

    with open(input_file, 'r', encoding='utf-8') as f:
        data = json.load(f)
        original_texts = []
        back_translated_texts = []

        for item in data:

            original_text = item.get("text", "")
            translated_text = item.get("translated_text", "")
            back_translated = item.get("back_translated_text", "")
            original_texts.append(original_text)
            back_translated_texts.append(back_translated)
        
        bleu = sacrebleu.corpus_bleu(back_translated_texts, [original_texts])
        bleu_score = bleu.score
        print(f"Back-translation terminée. Fichier généré : {output_file}")
        print(f"Score BLEU (corpus) = {bleu_score:.2f}")

if __name__ == "__main__":
    main()
    

            

Back-translation terminée. Fichier généré : backtranslated_3.5.json
Score BLEU (corpus) = 48.07


un score BLEU autour de 40 pour de la back-translation indique généralement une bonne préservation du sens et des structures principales.

Si vous aviez un score BLEU très faible (ex. < 20), ça pourrait indiquer que la traduction ou la back-traduction s’est grandement éloignée de l’original.

À l’inverse, un score élevé (ex. 70+) signifierait que la back-traduction est extrêmement proche de la phrase d’origine sur le plan lexical. Cela peut être souhaitable, ou au contraire indiquer une traduction trop littérale.

In [ ]:
%pip install bert-score


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 1.4 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.3.2 -> 24.3.1
[notice] To update, run: pip3.10 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [7]:
import json
from bert_score import score

# On suppose que le fichier contenant l'original, la traduction et la back-traduction s'appelle:
input_file = "backtranslated_3.5.json"

def main():
    with open(input_file, "r", encoding="utf-8") as f:
        data = json.load(f)

    original_texts = []
    backtranslated_texts = []

    for item in data:
        original = item.get("text", "")
        backtrans = item.get("back_translated_text", "")

        # On stocke la version anglaise originale et la version back-traduite
        original_texts.append(original)
        backtranslated_texts.append(backtrans)

    # BERTScore nécessite que la référence et l'hypothèse soient dans la même langue (anglais, ici).
    # Par défaut, lang="en" => BERTScore utilise un modèle anglais
    P, R, F1 = score(backtranslated_texts, original_texts, lang="en")

    print(f"Nombre de segments évalués : {len(original_texts)}")
    print(f"Score BERTScore (moyenne) :")
    print(f"  - Precision: {P.mean():.4f}")
    print(f"  - Recall:    {R.mean():.4f}")
    print(f"  - F1:        {F1.mean():.4f}")

if __name__ == "__main__":
    main()


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Nombre de segments évalués : 150
Score BERTScore (moyenne) :
  - Precision: 0.9609
  - Recall:    0.9659
  - F1:        0.9633


- F1 est souvent utilisé comme score global (entre 0 et 1). Plus c’est proche de 1, plus les phrases sont considérées sémantiquement similaires.
- on peut aussi consulter la Precision et le Recall, pour voir comment les tokens s’alignent.